In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import tree
from sklearn.tree import export_graphviz
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition, datasets
from sklearn.pipeline import Pipeline

In [2]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [4]:
import glob

path = r'C:\\Users\\hugob\\Desktop\\dsi\\ep2\\csv'
all_files = glob.glob(path + "/*.csv")
df = None
li = []
i = 0
for filename in all_files:
    if i == 0:
        df = spark.read.format("csv").option("header", "true").load(filename)
        i = 1
    else:
        aux = spark.read.format("csv").option("header", "true").load(filename)
        df = df.union(aux)
    

In [6]:
from pyspark.sql.functions import mean as _mean, stddev as _stddev, col
from pyspark.sql.functions import unix_timestamp, lit
from pyspark.sql.functions import *

df.withColumn("DATE", to_date(unix_timestamp(df["DATE"], "MM/dd/yyyy").cast("timestamp")))

def app(data_ini, data_fim, coluna, op):
    dffil = df.filter(df["DATE"] >= lit(data_ini)).filter(df["DATE"] <= lit(data_fim)) 
    dados = dffil.select(_mean(col(coluna)).alias('mean'),_stddev(col(coluna)).alias('std')).collect()
    mean = dados[0]['mean']
    std = dados[0]['std']
    if op == 'Media':
        return(mean)
    else: 
        return(std)
    return 

In [ ]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import datetime

window = tk.Tk()
window.rowconfigure(0, minsize=50)
window.columnconfigure([0, 1, 2, 3], minsize=50)
# T = tk.Text(height=30, width=90, state='disabled')
T = tk.Text(state='disabled')

def criarWidgets():
    titulo_selecao_coluna = tk.Label(text="Selecione a coluna")
    titulo_selecao_coluna.grid(row=0, column=0, padx=(20, 20))
    coluna = ("DATE","LATITUDE","LONGITUDE","ELEVATION","NAME","TEMP","TEMP_ATTRIBUTES","DEWP","DEWP_ATTRIBUTES","SLP","SLP_ATTRIBUTES","STP","STP_ATTRIBUTES","VISIB","VISIB_ATTRIBUTES","WDSP","WDSP_ATTRIBUTES","MXSPD","GUST","MAX","MAX_ATTRIBUTES","MIN","MIN_ATTRIBUTES","PRCP","PRCP_ATTRIBUTES","SNDP","FRSHTT")
    coluna_selecionada = tk.StringVar()

    selecao_coluna = ttk.Combobox(window, textvariable=coluna_selecionada)
    selecao_coluna['values'] = coluna
    selecao_coluna['state'] = 'readonly'  # normal
    selecao_coluna.grid(row=1, column=0, padx=(20, 20))

    titulo_selecao_operacao = tk.Label(text="Selecione a operacao")
    titulo_selecao_operacao.grid(row=0, column=1, padx=(20, 20))

    operacao = ('Media', 'Desvio padrao')
    operacao_selecionada = tk.StringVar()

    selecao_operacao = ttk.Combobox(window, textvariable=operacao_selecionada)
    selecao_operacao['values'] = operacao
    selecao_operacao['state'] = 'readonly'  # normal
    selecao_operacao.grid(row=1, column=1, padx=(20, 20))

    dataInicialLabel = tk.Label(text="Data incial")  
    dataInicialLabel.grid( row=0, column=2, padx=(20, 20))  
  
    dataInicial = tk.StringVar()
    dataInicialEntry = ttk.Entry(window, width = 15, textvariable = dataInicial)
    dataInicialEntry.grid(row=1, column=2, padx=(20, 20))

    dataFinalLabel = tk.Label(text="Data final")  
    dataFinalLabel.grid(row=0, column=3, padx=(20, 20))  
  
    dataFinal = tk.StringVar()
    dataFinalEntry = ttk.Entry(window, width = 15, textvariable = dataFinal)
    dataFinalEntry.grid(row=1, column=3, padx=(20, 20))

    slogan = tk.Button(
                   text="Executar",
                   command= lambda:enviar_infos(dataInicialEntry.get(), dataFinalEntry.get(),selecao_coluna.get(),selecao_operacao.get()))
    slogan.grid(column=4, row=1, padx=(20, 20))

    resultadoLabel = tk.Label(text="Resultado:")  
    resultadoLabel.place(height=20,x=20, y=100)
    T.place(height=20, width=600,x=20, y=120)
    # T.place(relx = 0.5, rely = 0.95, anchor = 's')


def enviar_infos(data_ini, data_fim, coluna, op):
    # print(data_ini, data_fim, coluna, op)
    valido = True
    if validarData(data_ini) == False:
        messagebox.showerror("showerror", "Por favor verificar se a data inicial está de acordo com o formato yyyy-mm-dd e é uma data valida.")
        valido = False 
    if validarData(data_fim) == False:
        messagebox.showerror("showerror", "Por favor verificar se a data final está de acordo com o formato yyyy-mm-dd e é uma data valida.")
        valido = False 
    if coluna == '':
        messagebox.showerror("showerror", "Por favor selecionar uma coluna")
        valido = False 
    if op == '':
        messagebox.showerror("showerror", "Por favor selecionar uma operacao")
        valido = False
    if valido == True:
         inserirResultado(app(data_ini, data_fim, coluna, op))

def validarData(date_string):
    format = "%Y-%m-%d"
    try:
        datetime.datetime.strptime(date_string, format)
        return True
    except ValueError:
        return False

def inserirResultado(resultado):
    T.configure(state='normal')
    T.delete('1.0', tk.END)
    T.insert(tk.END, resultado)
    T.configure(state='disabled')

window.geometry("800x600")
criarWidgets()
window.mainloop()


